In [1]:
#supress warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import sys
import os
_stderr = sys.stderr
null = open(os.devnull,'wb')

In [2]:
pwd

'/fast/AG_Bunina/Yusuf/jobs'

In [3]:
cd '/fast/AG_Bunina/Yusuf/Project_Endothelial_and_Stroke/Datasets/Chromatin_and_Gene_Exp/2023_Zhu_et_al/Zhu_et_al_2023/Python_Project/ScenicPlus/Zhu_et_al'

/fast/AG_Bunina/Yusuf/Project_Endothelial_and_Stroke/Datasets/Chromatin_and_Gene_Exp/2023_Zhu_et_al/Zhu_et_al_2023/Python_Project/ScenicPlus/Zhu_et_al


In [4]:
pwd

'/fast/AG_Bunina/Yusuf/Project_Endothelial_and_Stroke/Datasets/Chromatin_and_Gene_Exp/2023_Zhu_et_al/Zhu_et_al_2023/Python_Project/ScenicPlus/Zhu_et_al'

In [5]:
import scanpy as sc
#set some figure parameters for nice display inside jupyternotebooks.
%matplotlib inline
sc.settings.set_figure_params(dpi=80, frameon=False, figsize=(5, 5), facecolor='white')

In [6]:
work_dir = 'Zhu_et_al'

In [7]:
adata = sc.read_h5ad(os.path.join(work_dir, 'scRNA/adata_processed.h5ad'))


In [8]:
adata

AnnData object with n_obs × n_vars = 44330 × 30109
    obs: 'author_cell_type', 'age_group', 'donor_id', 'nCount_RNA', 'nFeature_RNA', 'nCount_ATAC', 'nFeature_ATAC', 'TSS_percentile', 'nucleosome_signal', 'percent_mt', 'assay_ontology_term_id', 'cell_type_ontology_term_id', 'development_stage_ontology_term_id', 'disease_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'organism_ontology_term_id', 'sex_ontology_term_id', 'tissue_ontology_term_id', 'suspension_type', 'is_primary_data', 'batch', 'tissue_type', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'observation_joinid', 'n_genes', 'doublet_score', 'predicted_doublet', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'celltype'
    var: 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype', 'feature_length', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly

In [9]:
adata.var_names

Index(['ENSG00000238009', 'ENSG00000241860', 'ENSG00000229905',
       'ENSG00000237491', 'ENSG00000177757', 'ENSG00000228794',
       'ENSG00000225880', 'ENSG00000230368', 'ENSG00000272438',
       'ENSG00000230699',
       ...
       'ENSG00000274847', 'ENSG00000276256', 'ENSG00000278198',
       'ENSG00000273496', 'ENSG00000278384', 'ENSG00000273748',
       'ENSG00000271254', 'ENSG00000276345', 'ENSG00000278817',
       'ENSG00000277196'],
      dtype='object', name='gene_ids', length=30109)

In [13]:

ensembl_version_dict = {'104': 'http://may2021.archive.ensembl.org/'}

import pybiomart as pbm

dataset = pbm.Dataset(name='hsapiens'+'_gene_ensembl',  host= ensembl_version_dict['104'])


annot = dataset.query(attributes=['chromosome_name', 'transcription_start_site', 'strand', 'external_gene_name', 'transcript_biotype', 'ensembl_gene_id'])
annot.columns = ['Chromosome', 'Start', 'Strand', 'Gene', 'Transcript_type', 'ensembl_id']
annot['Chromosome'] = annot['Chromosome'].astype('str')
filter = annot['Chromosome'].str.contains('CHR|GL|JH|MT')
annot = annot[~filter]
annot.columns=['Chromosome', 'Start', 'Strand', 'Gene', 'Transcript_type', 'ensembl_id']


In [14]:
annot

,Chromosome,Start,Strand,Gene,Transcript_type,ensembl_id
37,KI270711.1,24650,-1,NaN,protein_coding,ENSG00000271254
38,KI270711.1,29626,-1,NaN,protein_coding,ENSG00000271254
40,KI270442.1,380608,1,5S_rRNA,rRNA,ENSG00000278457
53,KI270442.1,217401,-1,5_8S_rRNA,rRNA,ENSG00000278294
656,KI270734.1,59711,1,NaN,pseudogene,ENSG00000278573
...,...,...,...,...,...,...
236041,1,36482899,-1,CSF3R,processed_transcript,ENSG00000119535
236042,1,36482934,-1,CSF3R,retained_intron,ENSG00000119535
236043,1,36482051,-1,CSF3R,protein_coding,ENSG00000119535
236044,1,36323645,-1,EVA1B,protein_coding,ENSG00000142694


In [23]:
# Keep only rows with unique Ensembl IDs
filtered_annot_unique = annot.drop_duplicates(subset='ensembl_id')

In [24]:
filtered_annot_unique

,Chromosome,Start,Strand,Gene,Transcript_type,ensembl_id
37,KI270711.1,24650,-1,NaN,protein_coding,ENSG00000271254
40,KI270442.1,380608,1,5S_rRNA,rRNA,ENSG00000278457
53,KI270442.1,217401,-1,5_8S_rRNA,rRNA,ENSG00000278294
656,KI270734.1,59711,1,NaN,pseudogene,ENSG00000278573
661,KI270744.1,51114,-1,U6,snRNA,ENSG00000278625
...,...,...,...,...,...,...
236014,1,36422181,-1,OSCP1,protein_coding,ENSG00000116885
236026,1,36418578,-1,SNORA63C,snoRNA,ENSG00000201448
236027,1,36464384,-1,MRPS15,protein_coding,ENSG00000116898
236031,1,36482913,-1,CSF3R,protein_coding,ENSG00000119535


In [25]:
filtered_annot_unique = filtered_annot_unique[filtered_annot_unique['ensembl_id'].isin(adata.var_names)]

In [26]:
filtered_annot_unique

,Chromosome,Start,Strand,Gene,Transcript_type,ensembl_id
37,KI270711.1,24650,-1,NaN,protein_coding,ENSG00000271254
662,KI270734.1,161852,-1,NaN,protein_coding,ENSG00000277196
718,KI270734.1,131494,1,NaN,protein_coding,ENSG00000278817
3604,KI270721.1,2585,1,NaN,protein_coding,ENSG00000276345
8959,Y,25394719,-1,LINC00266-4P,lncRNA,ENSG00000228786
...,...,...,...,...,...,...
236010,1,36397887,-1,LSM10,processed_transcript,ENSG00000181817
236014,1,36422181,-1,OSCP1,protein_coding,ENSG00000116885
236027,1,36464384,-1,MRPS15,protein_coding,ENSG00000116898
236031,1,36482913,-1,CSF3R,protein_coding,ENSG00000119535


In [27]:
adata.var_names

Index(['ENSG00000238009', 'ENSG00000241860', 'ENSG00000229905',
       'ENSG00000237491', 'ENSG00000177757', 'ENSG00000228794',
       'ENSG00000225880', 'ENSG00000230368', 'ENSG00000272438',
       'ENSG00000230699',
       ...
       'ENSG00000274847', 'ENSG00000276256', 'ENSG00000278198',
       'ENSG00000273496', 'ENSG00000278384', 'ENSG00000273748',
       'ENSG00000271254', 'ENSG00000276345', 'ENSG00000278817',
       'ENSG00000277196'],
      dtype='object', name='gene_ids', length=30109)

In [28]:
filtered_annot_unique.ensembl_id.tolist()[0:5]

['ENSG00000271254',
 'ENSG00000277196',
 'ENSG00000278817',
 'ENSG00000276345',
 'ENSG00000228786']

In [29]:
adata_filtered = adata[:, filtered_annot_unique.ensembl_id.tolist()]

In [30]:
adata_filtered

View of AnnData object with n_obs × n_vars = 44330 × 30088
    obs: 'author_cell_type', 'age_group', 'donor_id', 'nCount_RNA', 'nFeature_RNA', 'nCount_ATAC', 'nFeature_ATAC', 'TSS_percentile', 'nucleosome_signal', 'percent_mt', 'assay_ontology_term_id', 'cell_type_ontology_term_id', 'development_stage_ontology_term_id', 'disease_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'organism_ontology_term_id', 'sex_ontology_term_id', 'tissue_ontology_term_id', 'suspension_type', 'is_primary_data', 'batch', 'tissue_type', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'observation_joinid', 'n_genes', 'doublet_score', 'predicted_doublet', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'celltype'
    var: 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype', 'feature_length', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts',

In [31]:
adata_filtered.var_names

Index(['ENSG00000271254', 'ENSG00000277196', 'ENSG00000278817',
       'ENSG00000276345', 'ENSG00000228786', 'ENSG00000286217',
       'ENSG00000277930', 'ENSG00000114374', 'ENSG00000198692',
       'ENSG00000286009',
       ...
       'ENSG00000160679', 'ENSG00000143553', 'ENSG00000214193',
       'ENSG00000286379', 'ENSG00000196182', 'ENSG00000181817',
       'ENSG00000116885', 'ENSG00000116898', 'ENSG00000119535',
       'ENSG00000142694'],
      dtype='object', name='gene_ids', length=30088)

In [32]:
filtered_annot_unique

,Chromosome,Start,Strand,Gene,Transcript_type,ensembl_id
37,KI270711.1,24650,-1,NaN,protein_coding,ENSG00000271254
662,KI270734.1,161852,-1,NaN,protein_coding,ENSG00000277196
718,KI270734.1,131494,1,NaN,protein_coding,ENSG00000278817
3604,KI270721.1,2585,1,NaN,protein_coding,ENSG00000276345
8959,Y,25394719,-1,LINC00266-4P,lncRNA,ENSG00000228786
...,...,...,...,...,...,...
236010,1,36397887,-1,LSM10,processed_transcript,ENSG00000181817
236014,1,36422181,-1,OSCP1,protein_coding,ENSG00000116885
236027,1,36464384,-1,MRPS15,protein_coding,ENSG00000116898
236031,1,36482913,-1,CSF3R,protein_coding,ENSG00000119535


In [33]:
type(filtered_annot_unique)

pandas.core.frame.DataFrame

In [36]:
import pandas as pd

# Assuming adata_filtered.var_names is a list-like object
# and ensembl_ids is a column in your DataFrame

# Step 1: Extract the ensembl_ids column from your DataFrame
ensembl_ids_column = filtered_annot_unique['ensembl_id']

# Step 2: Convert adata_filtered.var_names to a list
adata_var_names_list = list(adata_filtered.var_names)

# Step 3: Compare the equality and order of the two lists
are_equal_and_same_order = (ensembl_ids_column.tolist() == adata_var_names_list)


In [37]:
are_equal_and_same_order

True

In [39]:
type(adata.var_names)

pandas.core.indexes.base.Index

In [40]:
type(filtered_annot_unique['Gene'])

pandas.core.series.Series

In [42]:
adata_filtered.var_names = filtered_annot_unique['Gene'].tolist()

In [43]:
adata_filtered.var_names

Index([           nan,            nan,            nan,            nan,
       'LINC00266-4P',    'LINC00279',            nan,        'USP9Y',
             'EIF1AY',            nan,
       ...
              'CHTOP',       'SNAPIN',       'SH3D21',            nan,
              'STK40',        'LSM10',        'OSCP1',       'MRPS15',
              'CSF3R',        'EVA1B'],
      dtype='object', length=30088)

In [44]:
import numpy as np

In [46]:
# Assuming adata_filtered.var_names is a list-like object

# Convert var_names to a set
var_names_set = set(adata_filtered.var_names)

# Check if the length of the set is the same as the length of the original list
if len(var_names_set) == len(adata_filtered.var_names):
    print("All elements in adata_filtered.var_names are unique.")
else:
    print("adata_filtered.var_names contains duplicate elements.")


adata_filtered.var_names contains duplicate elements.


In [47]:
len(var_names_set)

21315

In [48]:
len(adata_filtered.var_names)

30088

In [49]:
adata_filtered.var_names_make_unique()

TypeError: unsupported operand type(s) for +: 'float' and 'str'

In [50]:
# Convert all elements in var_names to string type
adata_filtered.var_names = adata_filtered.var_names.astype(str)

# Make var_names unique
adata_filtered.var_names_make_unique()


In [51]:
adata_filtered.var_names

Index(['nan', 'nan-1', 'nan-2', 'nan-3', 'LINC00266-4P', 'LINC00279', 'nan-4',
       'USP9Y', 'EIF1AY', 'nan-5',
       ...
       'CHTOP', 'SNAPIN', 'SH3D21', 'nan-8765', 'STK40', 'LSM10', 'OSCP1',
       'MRPS15', 'CSF3R', 'EVA1B'],
      dtype='object', length=30088)

In [52]:
adata_filtered.write(os.path.join(work_dir, 'scRNA/adata_filtered_unique_processed.h5ad'), compression='gzip')

In [53]:
import session_info

In [54]:
session_info.show()

In [55]:
adata_filtered

AnnData object with n_obs × n_vars = 44330 × 30088
    obs: 'author_cell_type', 'age_group', 'donor_id', 'nCount_RNA', 'nFeature_RNA', 'nCount_ATAC', 'nFeature_ATAC', 'TSS_percentile', 'nucleosome_signal', 'percent_mt', 'assay_ontology_term_id', 'cell_type_ontology_term_id', 'development_stage_ontology_term_id', 'disease_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'organism_ontology_term_id', 'sex_ontology_term_id', 'tissue_ontology_term_id', 'suspension_type', 'is_primary_data', 'batch', 'tissue_type', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'observation_joinid', 'n_genes', 'doublet_score', 'predicted_doublet', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'celltype'
    var: 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype', 'feature_length', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly

In [58]:
adata_filtered.raw.to_adata()

AnnData object with n_obs × n_vars = 44330 × 30109
    obs: 'author_cell_type', 'age_group', 'donor_id', 'nCount_RNA', 'nFeature_RNA', 'nCount_ATAC', 'nFeature_ATAC', 'TSS_percentile', 'nucleosome_signal', 'percent_mt', 'assay_ontology_term_id', 'cell_type_ontology_term_id', 'development_stage_ontology_term_id', 'disease_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'organism_ontology_term_id', 'sex_ontology_term_id', 'tissue_ontology_term_id', 'suspension_type', 'is_primary_data', 'batch', 'tissue_type', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'observation_joinid', 'n_genes', 'doublet_score', 'predicted_doublet', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'celltype'
    var: 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype', 'feature_length', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'
    uns:

In [59]:
adata_raw = adata_filtered.raw.to_adata().copy()

In [60]:
adata_raw

AnnData object with n_obs × n_vars = 44330 × 30109
    obs: 'author_cell_type', 'age_group', 'donor_id', 'nCount_RNA', 'nFeature_RNA', 'nCount_ATAC', 'nFeature_ATAC', 'TSS_percentile', 'nucleosome_signal', 'percent_mt', 'assay_ontology_term_id', 'cell_type_ontology_term_id', 'development_stage_ontology_term_id', 'disease_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'organism_ontology_term_id', 'sex_ontology_term_id', 'tissue_ontology_term_id', 'suspension_type', 'is_primary_data', 'batch', 'tissue_type', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'observation_joinid', 'n_genes', 'doublet_score', 'predicted_doublet', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'celltype'
    var: 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype', 'feature_length', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'
    uns:

In [61]:
annot

,Chromosome,Start,Strand,Gene,Transcript_type,ensembl_id
37,KI270711.1,24650,-1,NaN,protein_coding,ENSG00000271254
38,KI270711.1,29626,-1,NaN,protein_coding,ENSG00000271254
40,KI270442.1,380608,1,5S_rRNA,rRNA,ENSG00000278457
53,KI270442.1,217401,-1,5_8S_rRNA,rRNA,ENSG00000278294
656,KI270734.1,59711,1,NaN,pseudogene,ENSG00000278573
...,...,...,...,...,...,...
236041,1,36482899,-1,CSF3R,processed_transcript,ENSG00000119535
236042,1,36482934,-1,CSF3R,retained_intron,ENSG00000119535
236043,1,36482051,-1,CSF3R,protein_coding,ENSG00000119535
236044,1,36323645,-1,EVA1B,protein_coding,ENSG00000142694


In [62]:
# Keep only rows with unique Ensembl IDs
filtered_annot_unique = annot.drop_duplicates(subset='ensembl_id')

In [63]:
filtered_annot_unique

,Chromosome,Start,Strand,Gene,Transcript_type,ensembl_id
37,KI270711.1,24650,-1,NaN,protein_coding,ENSG00000271254
40,KI270442.1,380608,1,5S_rRNA,rRNA,ENSG00000278457
53,KI270442.1,217401,-1,5_8S_rRNA,rRNA,ENSG00000278294
656,KI270734.1,59711,1,NaN,pseudogene,ENSG00000278573
661,KI270744.1,51114,-1,U6,snRNA,ENSG00000278625
...,...,...,...,...,...,...
236014,1,36422181,-1,OSCP1,protein_coding,ENSG00000116885
236026,1,36418578,-1,SNORA63C,snoRNA,ENSG00000201448
236027,1,36464384,-1,MRPS15,protein_coding,ENSG00000116898
236031,1,36482913,-1,CSF3R,protein_coding,ENSG00000119535


In [64]:
adata_raw.var_names

Index(['ENSG00000238009', 'ENSG00000241860', 'ENSG00000229905',
       'ENSG00000237491', 'ENSG00000177757', 'ENSG00000228794',
       'ENSG00000225880', 'ENSG00000230368', 'ENSG00000272438',
       'ENSG00000230699',
       ...
       'ENSG00000274847', 'ENSG00000276256', 'ENSG00000278198',
       'ENSG00000273496', 'ENSG00000278384', 'ENSG00000273748',
       'ENSG00000271254', 'ENSG00000276345', 'ENSG00000278817',
       'ENSG00000277196'],
      dtype='object', name='gene_ids', length=30109)

In [68]:
filtered_annot_unique = filtered_annot_unique[filtered_annot_unique['ensembl_id'].isin(adata_raw.var_names)]

In [69]:
filtered_annot_unique

,Chromosome,Start,Strand,Gene,Transcript_type,ensembl_id
37,KI270711.1,24650,-1,NaN,protein_coding,ENSG00000271254
662,KI270734.1,161852,-1,NaN,protein_coding,ENSG00000277196
718,KI270734.1,131494,1,NaN,protein_coding,ENSG00000278817
3604,KI270721.1,2585,1,NaN,protein_coding,ENSG00000276345
8959,Y,25394719,-1,LINC00266-4P,lncRNA,ENSG00000228786
...,...,...,...,...,...,...
236010,1,36397887,-1,LSM10,processed_transcript,ENSG00000181817
236014,1,36422181,-1,OSCP1,protein_coding,ENSG00000116885
236027,1,36464384,-1,MRPS15,protein_coding,ENSG00000116898
236031,1,36482913,-1,CSF3R,protein_coding,ENSG00000119535


In [70]:
adata_RAW_filtered = adata_raw[:, filtered_annot_unique.ensembl_id.tolist()]

In [71]:
adata_RAW_filtered

View of AnnData object with n_obs × n_vars = 44330 × 30088
    obs: 'author_cell_type', 'age_group', 'donor_id', 'nCount_RNA', 'nFeature_RNA', 'nCount_ATAC', 'nFeature_ATAC', 'TSS_percentile', 'nucleosome_signal', 'percent_mt', 'assay_ontology_term_id', 'cell_type_ontology_term_id', 'development_stage_ontology_term_id', 'disease_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'organism_ontology_term_id', 'sex_ontology_term_id', 'tissue_ontology_term_id', 'suspension_type', 'is_primary_data', 'batch', 'tissue_type', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'observation_joinid', 'n_genes', 'doublet_score', 'predicted_doublet', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'celltype'
    var: 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype', 'feature_length', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'


In [72]:
filtered_annot_unique

,Chromosome,Start,Strand,Gene,Transcript_type,ensembl_id
37,KI270711.1,24650,-1,NaN,protein_coding,ENSG00000271254
662,KI270734.1,161852,-1,NaN,protein_coding,ENSG00000277196
718,KI270734.1,131494,1,NaN,protein_coding,ENSG00000278817
3604,KI270721.1,2585,1,NaN,protein_coding,ENSG00000276345
8959,Y,25394719,-1,LINC00266-4P,lncRNA,ENSG00000228786
...,...,...,...,...,...,...
236010,1,36397887,-1,LSM10,processed_transcript,ENSG00000181817
236014,1,36422181,-1,OSCP1,protein_coding,ENSG00000116885
236027,1,36464384,-1,MRPS15,protein_coding,ENSG00000116898
236031,1,36482913,-1,CSF3R,protein_coding,ENSG00000119535


In [73]:
import pandas as pd

# Assuming adata_filtered.var_names is a list-like object
# and ensembl_ids is a column in your DataFrame

# Step 1: Extract the ensembl_ids column from your DataFrame
ensembl_ids_column = filtered_annot_unique['ensembl_id']

# Step 2: Convert adata_filtered.var_names to a list
adata_var_names_list = list(adata_RAW_filtered.var_names)

# Step 3: Compare the equality and order of the two lists
are_equal_and_same_order = (ensembl_ids_column.tolist() == adata_var_names_list)

In [74]:
are_equal_and_same_order

True

In [75]:
adata_RAW_filtered.var_names = filtered_annot_unique['Gene'].tolist()

In [77]:
adata_RAW_filtered.var_names

Index([           nan,            nan,            nan,            nan,
       'LINC00266-4P',    'LINC00279',            nan,        'USP9Y',
             'EIF1AY',            nan,
       ...
              'CHTOP',       'SNAPIN',       'SH3D21',            nan,
              'STK40',        'LSM10',        'OSCP1',       'MRPS15',
              'CSF3R',        'EVA1B'],
      dtype='object', length=30088)

In [ ]:
# In Python, set object can only contain unique elements. If any object is converted to a set, any duplicate elements are removed.
# If the 'length" of before and after set conversion are identical, it means that all elements are unique. 

# In Python, we can use list() and set(/) functions to convert var_names and obs_names to a set and a list, respectively.

In [82]:
import numpy as np

# Assuming adata_filtered.var_names is a list-like object

# Convert var_names to a set
var_names_set = set(adata_RAW_filtered.var_names)

# Check if the length of the set is the same as the length of the original list
if len(var_names_set) == len(adata_RAW_filtered.var_names):
    print("All elements in adata_RAW_filtered.var_names are unique.")
else:
    print("adata_RAW_filtered.var_names contains duplicate elements.")


adata_RAW_filtered.var_names contains duplicate elements.


In [80]:
len(var_names_set)

21315

In [81]:
len(adata_RAW_filtered.var_names)

30088

In [83]:
# Convert all elements in var_names to string type
adata_RAW_filtered.var_names = adata_RAW_filtered.var_names.astype(str)

# Make var_names unique
adata_RAW_filtered.var_names_make_unique()

In [84]:
adata_RAW_filtered.var_names

Index(['nan', 'nan-1', 'nan-2', 'nan-3', 'LINC00266-4P', 'LINC00279', 'nan-4',
       'USP9Y', 'EIF1AY', 'nan-5',
       ...
       'CHTOP', 'SNAPIN', 'SH3D21', 'nan-8765', 'STK40', 'LSM10', 'OSCP1',
       'MRPS15', 'CSF3R', 'EVA1B'],
      dtype='object', length=30088)

In [85]:
import numpy as np

# Assuming adata_filtered.var_names is a list-like object

# Convert var_names to a set
var_names_set = set(adata_RAW_filtered.var_names)

# Check if the length of the set is the same as the length of the original list
if len(var_names_set) == len(adata_RAW_filtered.var_names):
    print("All elements in adata_RAW_filtered.var_names are unique.")
else:
    print("adata_RAW_filtered.var_names contains duplicate elements.")

All elements in adata_RAW_filtered.var_names are unique.


In [86]:
adata_RAW_filtered.write(os.path.join(work_dir, 'scRNA/adata_filtered_unique_names_RAW.h5ad'), compression='gzip')

In [87]:
session_info.show()